In [1]:
import boto3

In [2]:
s3 = boto3.client("s3")

In [3]:
s3

In [4]:
response = s3.list_objects_v2(Bucket="llm-finetune-dataset-g1", Prefix="datasets/")

In [5]:
for obj in response.get("Contents", []):
    print(obj)

{'Key': 'datasets/', 'LastModified': datetime.datetime(2025, 11, 13, 20, 27, 3, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'datasets/pharma_instruction_data.csv', 'LastModified': datetime.datetime(2025, 11, 14, 16, 52, 9, tzinfo=tzlocal()), 'ETag': '"7a3bbfb7acae4d8dc88dc1a08501d745"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 2469, 'StorageClass': 'STANDARD'}


In [13]:
!which python

/home/ec2-user/anaconda3/envs/python3/bin/python


In [7]:
dataset_path = "s3://llm-finetune-dataset-g1/datasets/pharma_instruction_data.csv/"

In [1]:
from datasets import load_dataset

ModuleNotFoundError: No module named 'datasets'

In [2]:
dataset = load_dataset("csv", data_files={"train":dataset_path}, split="train")

NameError: name 'load_dataset' is not defined

In [ ]:
dataset

In [ ]:
print(dataset[1])

In [ ]:
def format_example(example):
    prompt = f"## Instruction:\n {example['instruction']}\n## Input: \n{example['input']}\n ## Response:\n{example['output']}"
    return {"text": prompt}

In [ ]:
dataset = dataset.map(format_example)

In [ ]:
dataset

In [ ]:
dataset['text'][0]

In [ ]:
!pip install transformers

In [ ]:
!pip install peft

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
!pip install bitsandbytes==0.43.3 peft==0.11.1 s3fs==2024.3.1

In [ ]:
model_id = "Tinylama/TinyLlama-1.1B-intermediate-sstep-1431k-3T"

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

NameError: name 'AutoTokenizer' is not defined

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize_fn(example):
    tokens = tokenizer(example["text"], truncation=True, padding = max_length, max_length= 512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [ ]:
tokenized = dataset.map(tokenize_fn, batched=True)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

In [ ]:
!pip install torch

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules["q_proj", "v_proj"],
    bias="none"
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
args = TrainingArguments(
    output_dir="./tinyllama-instruction",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumilation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_total_limit=1,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
)

In [ ]:
trainer.train()

In [ ]:
model_path="/content/tinyllama-instruction/checkpoint-3"

In [ ]:
instruction_model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

In [ ]:
prompt = "Explain the mechanism of action of Metformin."

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [ ]:
outputs = instruction_model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1
)

In [ ]:
print("\nModel Output: \n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))